In [58]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary

In [59]:
df = pd.read_parquet("~/Data/BOQ_data.parquet")

df

,A2ME,BEME,BEME_adj,BEME_ind,CEI,DATE,DEC_ME,DEC_SHROUT,Div,Div_ann,...,s2invt,s2rect,sale,sale_ind,sales_g,sga2s,spread_mean,std_turn,std_vol,suv
0,NaN,NaN,NaN,0.824752,NaN,1986-01-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,0.825905,NaN,1986-02-28,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.076998,0.212080,7.804561e+03,NaN
2,NaN,NaN,NaN,0.825905,NaN,1986-03-31,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.055511,0.107977,3.973568e+03,-0.363314
3,NaN,NaN,NaN,0.825015,NaN,1986-04-30,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.037231,0.174533,6.422826e+03,-0.571056
4,NaN,NaN,NaN,0.824672,NaN,1986-05-31,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.048336,0.150228,5.527503e+03,-0.708679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568650,0.211263,0.114912,-0.746238,0.861151,-12.189408,2023-08-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000152,0.708916,2.250092e+07,-1.580970
3568651,0.211263,0.114912,-0.750723,0.865635,-11.613183,2023-09-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000133,0.465122,1.476294e+07,-0.371920
3568652,0.211263,0.114912,-0.731231,0.846143,-12.752123,2023-10-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000177,0.531342,1.686876e+07,0.809301
3568653,0.211263,0.114912,-0.731231,0.846143,-7.943286,2023-11-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000245,0.614813,1.954419e+07,0.278301


In [60]:
df.columns

Index(['A2ME', 'BEME', 'BEME_adj', 'BEME_ind', 'CEI', 'DATE', 'DEC_ME',
       'DEC_SHROUT', 'Div', 'Div_ann', 'E2P', 'EPS', 'IOR', 'LME', 'LME_ind',
       'LTurnover', 'ME', 'NumOwners', 'PERMNO', 'PRC', 'Q', 'RET', 'ROC',
       'S2P', 'SICCD', 'Turnover', 'VOL', 'at', 'at_adj', 'at_ind', 'ato',
       'be', 'beta', 'beta_daily', 'beta_sq', 'c', 'cash_lt', 'cto',
       'cum_return_12_2', 'cum_return_12_7', 'cum_return_1_0',
       'cum_return_36_13', 'cum_return_60_1', 'cum_return_6_2', 'd2a', 'd_ceq',
       'dpi2a', 'fc2y', 'free_cf', 'idio_vol', 'investment', 'lev', 'noa',
       'oa', 'ol', 'p1_hml_beta', 'p1_intercept', 'p1_mktrf_beta', 'p1_n',
       'p1_smb_beta', 'p3_intercept', 'p3_lmktrf_beta', 'p3_mktrf_beta',
       'p3_n', 'pcm', 'pm', 'pm_adj', 'prof', 'rel_to_high_price', 'ret_max',
       'rna', 'roa', 'roe', 'roic', 's2c', 's2invt', 's2rect', 'sale',
       'sale_ind', 'sales_g', 'sga2s', 'spread_mean', 'std_turn', 'std_vol',
       'suv'],
      dtype='object')

In [61]:
keep = ['DATE', 'PERMNO', 'BEME', 'BEME_adj', 'PRC', 'RET']
df = df[keep]

df = df.rename(columns={'DATE':'caldt','PERMNO':'permno','BEME':'beme', 'BEME_adj':'beme_adj', 'PRC':'prc', 'RET': 'ret'})

df = df.sort_values(['permno','caldt'])

df['mdt'] = df['caldt'].dt.strftime("%Y-%m")

df = df.reset_index(drop=True)

df

,caldt,permno,beme,beme_adj,prc,ret,mdt
0,1986-01-31,10000.0,NaN,NaN,4.375000,NaN,1986-01
1,1986-02-28,10000.0,NaN,NaN,3.250000,-0.257143,1986-02
2,1986-03-31,10000.0,NaN,NaN,4.437500,0.365385,1986-03
3,1986-04-30,10000.0,NaN,NaN,4.000000,-0.098592,1986-04
4,1986-05-31,10000.0,NaN,NaN,3.109375,-0.222656,1986-05
...,...,...,...,...,...,...,...
3534200,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08
3534201,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09
3534202,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10
3534203,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11


In [62]:
df['log_ret'] = np.log(1 + df['ret'])

df['ret_1m'] = df.groupby('permno').rolling(1,1)['log_ret'].sum().reset_index(drop=True)
df['ret_1m'] = df.groupby('permno')['ret_1m'].shift(-1)

df

,caldt,permno,beme,beme_adj,prc,ret,mdt,log_ret,ret_1m
0,1986-01-31,10000.0,NaN,NaN,4.375000,NaN,1986-01,NaN,-0.297252
1,1986-02-28,10000.0,NaN,NaN,3.250000,-0.257143,1986-02,-0.297252,0.311436
2,1986-03-31,10000.0,NaN,NaN,4.437500,0.365385,1986-03,0.311436,-0.103797
3,1986-04-30,10000.0,NaN,NaN,4.000000,-0.098592,1986-04,-0.103797,-0.251873
4,1986-05-31,10000.0,NaN,NaN,3.109375,-0.222656,1986-05,-0.251873,-0.005038
...,...,...,...,...,...,...,...,...,...
3534200,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,-0.035588,-0.030929
3534201,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,-0.030929,-0.219832
3534202,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,-0.219832,0.178464
3534203,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.178464,0.034390


In [63]:
df['prc_lag'] = df.groupby('permno')['prc'].shift(1)

df = df.query("beme == beme and prc_lag >= 5").reset_index(drop=True)

df

,caldt,permno,beme,beme_adj,prc,ret,mdt,log_ret,ret_1m,prc_lag
0,1987-07-31,10001.0,1.014415,-0.008146,6.000000,0.021277,1987-07,0.021053,0.080043,5.875000
1,1987-08-31,10001.0,1.014415,-0.008146,6.500000,0.083333,1987-08,0.080043,-0.022560,6.000000
2,1987-09-30,10001.0,1.014415,-0.008146,6.250000,-0.022308,1987-09,-0.022560,0.019803,6.500000
3,1987-10-31,10001.0,1.014415,-0.008146,6.375000,0.020000,1987-10,0.019803,-0.029853,6.250000
4,1987-11-30,10001.0,1.014415,-0.008146,6.187500,-0.029412,1987-11,-0.029853,-0.034111,6.375000
...,...,...,...,...,...,...,...,...,...,...
2086652,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,-0.035588,-0.030929,267.429993
2086653,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,-0.030929,-0.219832,258.079987
2086654,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,-0.219832,0.178464,250.220001
2086655,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.178464,0.034390,200.839996


In [64]:
df['beme_bins'] = df.groupby("mdt")['beme'].transform(lambda x: pd.qcut(x, 10, labels=False))

df

,caldt,permno,beme,beme_adj,prc,ret,mdt,log_ret,ret_1m,prc_lag,beme_bins
0,1987-07-31,10001.0,1.014415,-0.008146,6.000000,0.021277,1987-07,0.021053,0.080043,5.875000,8
1,1987-08-31,10001.0,1.014415,-0.008146,6.500000,0.083333,1987-08,0.080043,-0.022560,6.000000,7
2,1987-09-30,10001.0,1.014415,-0.008146,6.250000,-0.022308,1987-09,-0.022560,0.019803,6.500000,7
3,1987-10-31,10001.0,1.014415,-0.008146,6.375000,0.020000,1987-10,0.019803,-0.029853,6.250000,8
4,1987-11-30,10001.0,1.014415,-0.008146,6.187500,-0.029412,1987-11,-0.029853,-0.034111,6.375000,8
...,...,...,...,...,...,...,...,...,...,...,...
2086652,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,-0.035588,-0.030929,267.429993,0
2086653,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,-0.030929,-0.219832,258.079987,0
2086654,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,-0.219832,0.178464,250.220001,0
2086655,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.178464,0.034390,200.839996,0


In [65]:
# Portfolio df for summary stats
port = df.groupby(['caldt', 'beme_bins'])['ret_1m'].mean().unstack(level=['beme_bins'])*100

# Drop rows from port where we don't know the holding return (most recent 5 years)
port = port.dropna()

# Spread portfolios
port['spread'] = port[9] - port[0]

port.head()

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
caldt,,,,,,,,,,,
1956-07-31,-2.400411,-2.548842,-2.915443,-2.232052,-2.837254,0.107752,-0.788362,-1.812984,-0.344681,-1.063068,1.337343
1956-08-31,-6.060851,-7.067093,-5.617036,-4.275576,-4.100774,-3.138147,-3.825779,-3.566795,-2.861245,-2.240838,3.820013
1956-09-30,0.664211,1.463392,0.323620,1.356248,1.708495,1.529263,0.736612,-0.058453,0.887656,1.418277,0.754066
1956-10-31,-1.932929,-0.721428,0.486229,0.518967,1.755439,0.371152,0.686015,0.469530,0.648072,1.879056,3.811985
1956-11-30,4.736111,3.201574,3.446311,3.579786,4.115274,4.175386,3.559940,2.651208,2.375114,1.451963,-3.284148


In [66]:
summary(port).loc[['count','mean','std','tstat']].round(3)

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
count,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000
mean,-0.413,-0.068,0.172,0.328,0.428,0.530,0.587,0.614,0.746,0.685,1.098
std,7.023,6.257,5.741,5.555,5.358,5.195,5.000,5.055,5.012,5.551,4.414
tstat,-1.673,-0.311,0.851,1.680,2.273,2.901,3.337,3.453,4.236,3.510,7.078
